
<a href="https://colab.research.google.com/drive/1S153LgzXKIjiA3QjBrEB5h_dxztSPf4o?usp=sharing" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>



In [1]:
#1 Install libraries (if needed)
!pip install -q unsloth "torch>=2.1" transformers peft datasets trl accelerate py7zr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.2/193.2 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [10]:
# Main imports
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
from transformers import TrainingArguments
from trl import SFTTrainer

In [11]:
#2 Load TinyLlama-1.1B-Chat
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    max_seq_length=4096,
    dtype=torch.float16,
    load_in_4bit=True,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing=True,
    random_state=42,
)


==((====))==  Unsloth 2025.4.1: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: unsloth/tinyllama-chat-bnb-4bit can only handle sequence lengths of at most 2048.
But with kaiokendev's RoPE scaling of 2.0, it can be magically be extended to 4096!


In [3]:
#3 Load small IMDb subset (classification)
from datasets import load_dataset

imdb_dataset = load_dataset("imdb", split="train[:2000]")


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
#4 Load samsum dataset (small subset)
samsum_dataset = load_dataset("samsum", split="train[:1000]", trust_remote_code=True)

README.md:   0%|          | 0.00/7.04k [00:00<?, ?B/s]

samsum.py:   0%|          | 0.00/3.36k [00:00<?, ?B/s]

corpus.7z:   0%|          | 0.00/2.94M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

In [5]:
#5 Format Classification Dataset (OpenChat template)

def format_classification(example):
    sentiment = "Positive" if example["label"] == 1 else "Negative"
    return {
        "text": (
            f"<|im_start|>user\n"
            f"Classify the sentiment of this movie review:\n\n\"{example['text']}\"\n\n"
            f"Respond only with Positive or Negative.\n"
            f"<|im_end|>\n<|im_start|>assistant\n{sentiment}\n<|im_end|>"
        )
    }

formatted_classification = imdb_dataset.map(format_classification)
formatted_classification = formatted_classification.remove_columns(["label"])

# Format Conversational Dataset (ChatML template)

def format_conversation(example):
    return {
        "text": (
            f"<|im_start|>user\n{example['dialogue']}\n"
            f"<|im_end|>\n<|im_start|>assistant\n{example['summary']}\n<|im_end|>"
        )
    }

formatted_conversation = samsum_dataset.map(format_conversation)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [12]:
#6 Merge classification + conversation into a single dataset
from datasets import concatenate_datasets

multi_task_dataset = concatenate_datasets([formatted_classification, formatted_conversation])

# Shuffle merged dataset
multi_task_dataset = multi_task_dataset.shuffle(seed=42)

# Preview
print(multi_task_dataset[0])


{'text': "<|im_start|>user\nGemma: Forgot my keys :(\r\nSuze: Come by my office\r\nGemma: okay thanks!\r\nSuze: no problem\n<|im_end|>\n<|im_start|>assistant\nGemma will come by Suze's office as she forgot her keys.\n<|im_end|>", 'id': '13728796', 'dialogue': 'Gemma: Forgot my keys :(\r\nSuze: Come by my office\r\nGemma: okay thanks!\r\nSuze: no problem', 'summary': "Gemma will come by Suze's office as she forgot her keys."}


In [13]:
#7 Tokenize merged dataset
def tokenize(example):
    tokens = tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=4096, #Extended
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_dataset = multi_task_dataset.map(tokenize, remove_columns=["text"])

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [15]:
#8 Fine-tune using SFTTrainer

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="tinyllama_multi_task",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    num_train_epochs=1,
    max_steps=100,
    logging_steps=5,
    save_strategy="no",
    fp16=True,
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_dataset,
    dataset_text_field="text",
    max_seq_length=4096,
    args=training_args,
)

trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,000 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 2 x 1) = 4
 "-____-"     Trainable parameters = 4,505,600/4,000,000,000 (0.11% trained)


Step,Training Loss
5,2.574900
10,2.447700
15,2.463900
20,2.362100
25,2.428200
30,2.403100
35,2.273700
40,2.297900
45,2.306900
50,2.208300


TrainOutput(global_step=100, training_loss=2.2942074871063234, metrics={'train_runtime': 771.1896, 'train_samples_per_second': 0.519, 'train_steps_per_second': 0.13, 'total_flos': 1.02139623899136e+16, 'train_loss': 2.2942074871063234})

In [16]:
#9 Save model and tokenizer
model.save_pretrained("tinyllama_multi_task_model")
tokenizer.save_pretrained("tinyllama_multi_task_model")

('tinyllama_multi_task_model/tokenizer_config.json',
 'tinyllama_multi_task_model/special_tokens_map.json',
 'tinyllama_multi_task_model/tokenizer.model',
 'tinyllama_multi_task_model/added_tokens.json',
 'tinyllama_multi_task_model/tokenizer.json')

In [17]:
#10 Inference Testing

def multi_task_inference(prompt, max_tokens=100):
    formatted_prompt = (
        f"<|im_start|>user\n{prompt}\n"
        f"<|im_end|>\n<|im_start|>assistant\n"
    )
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id,
    )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded.split("<|im_start|>assistant\n")[-1].split("<|im_end|>")[0].strip()

# Test Classification Task
print(multi_task_inference("Classify the sentiment of this review: 'The movie had breathtaking scenes and strong performances.'"))

# Test Conversation Task
print(multi_task_inference("Hey, are you free this evening? Let's catch up over coffee!"))

Positive
Sure, here are a few coffee shops in the area:
1. Starbucks
2. Dunkin' Donuts
3. Coffee Bean & Tea Leaf
